# Código 15: Iluminação

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [47]:
#Bibliotecas
try:
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image
except ImportError:
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    !pip install pyglm
    !pip install pillow
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Multiplas Fontes"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

In [48]:
#GLSL para Vertex Shader
vertex_code = """
        //POSICAO VERTICES, TEXTURA E NORMAL
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
        //VALORES REPASSADOS PARA O SHADER FRAGMENT
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;

        //MATRIZES DE TRANSFORMACAO 
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(  model * vec4(position, 1.0));
            out_normal = vec3( model *vec4(normals, 1.0));            
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        // FONTE DE LUZ AMBIENTE
        vec3 lightColor = vec3(1.0, 1.0, 1.0);

        // POSICAO DA FONTE DE LUZ E COR DA LUZ 1
        uniform vec3 lightPos1; // define coordenadas de posicao da luz
        vec3 lightColor1 = vec3(1.0, 0.0, 0.0);

        // POSICAO DA FONTE DE LUZ E COR DA LUZ 2
        uniform vec3 lightPos2; // define coordenadas de posicao da luz
        vec3 lightColor2 = vec3(0.0, 1.0, 0.0);
        
        // ILUMINACAO AMBIENTE E DIFUSA
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // ILUMINACAO ESPECULAR
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular

        // VARIAVEIS VINDOS DO SHADER VERTEX
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader

        // PARAMETRO DE TEXTURA
        uniform sampler2D samplerTexture;      
        
        void main(){
        
            // REFLEXAO AMBIENTE
            vec3 ambient = ka * lightColor;  
            
            ////////////////////////
            // LUZ #1
            ////////////////////////
        
            // REFLEXAO DIFUSA
            vec3 norm1 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // direcao da luz
            float diff1 = max(dot(norm1, lightDir1), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse1 = kd * diff1 * lightColor1; // iluminacao difusa
            
            // REFLEXAO ESPECULAR
            vec3 viewDir1 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir1 = normalize(reflect(-lightDir1, norm1)); // direcao da reflexao
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ks * spec1 * lightColor1;       

            ////////////////////////
            // LUZ #2
            ////////////////////////

            // REFLEXAO DIFUSA
            vec3 norm2 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(lightPos2 - out_fragPos); // direcao da luz
            float diff2 = max(dot(norm2, lightDir2), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse2 = kd * diff2 * lightColor2; // iluminacao difusa
            
            // REFLEXAO ESPECULAR
            vec3 viewDir2 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir2 = normalize(reflect(-lightDir2, norm2)); // direcao da reflexao
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ks * spec2 * lightColor2;   

            ////////////////////////
            // COMBINANDO DUAS FONTES
            ////////////////////////
            
            // MODELO DE PHONG
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse1 + diffuse2 + specular1 + specular2),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;
        }
        """

### Solicitando espaço, compilando e linkando

In [49]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Extraindo os dados dos arquivos Wavefront

In [50]:
# Função: carrega o arquivo Wavefront
# Entrada: nome do arquivo
# Saida: estrutura que armazena o elemento (vertices, textura, normais e relations)
def load_model_from_file(file_path_name):
    # Arrays que armazenaram informações de coordenadas ou vetores
    vertices = []       # Posições dos vértices
    texture_coords = [] # Coordenada dos vértices de textura
    normals = []        # Coordenada que define os vetores normais
    relations = []      # Modelo que conecta, a partir da funções dadas no .obj, 
                        # vértices do objeto, vértices dde textura e os vetores normais

    # Não é utilizado, mas refere-se ao material do .obj
    material = None

    # Abre o arquivo obj (wavefront) para leitura
    for line in open(file_path_name, "r"): ## para cada linha do arquivo .obj
        # Se for comentário, ignore esta linha e use a próxima
        if line.startswith('#'): continue

        # Quebra a linha por espaço
        values = line.split()
        # Se não há informações na linha, ignore esta linha e use a próxima
        if not values: continue

        # Recupera as informações
        ### Armazena coordenadas dos vertices do elemento no vetor vertices
        if values[0] == 'v':
            vertices.append(values[1:4])
        ### Armazena vetor normais no vetor normals
        elif values[0] == 'vn':
            normals.append(values[1:4])
        ### Armazena coordenadas das texturas no vetor texture_coords
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])
        ### Define o material 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        ### Armazena informações sobre a construção das relations
        elif values[0] == 'f':
            # Declara vetores intermediários
            relation_vert = []
            relation_texture = []
            relation_normal = []
            # Para cada uma das triplas da linha que define a função
            for bloco in values[1:]:
                # Separa o elemento em vetor de elementos separando os números que são separados por /
                positions = bloco.split('/')
                # Adiciona o primeiro número no vetor relation_vert (que representa o número da linha que encontra-se um vértice)
                relation_vert.append(int(positions[0]))
                # Adiciona o terceiro número no vetor relation_normal (que representa o número da linha que encontra-se a normal para aquele vértice)
                relation_normal.append(int(positions[2]))
                # Se o vetor com elementos separados por / for maior ou igual que dois
                # Se o segundo número do elemento for maior do que zero
                if len(positions) >= 2 and len(positions[1]) > 0:
                    # Adicione o segundo número no vetor relation_texture (que representa o número da linha que encontra-se um vértice de textura da figura)
                    relation_texture.append(int(positions[1]))
                else:
                    # Se não for maior ou igual a dois ou não for maior que zero, coloque zero na textura
                    relation_texture.append(0)
            # Após conseguir, provavelmente, os três valores para vértice, os três valores para textura, os três valores para normal e o tipo de material, insira no vetor relations
            relations.append((relation_vert, relation_texture, relation_normal, material))

    # Armazena cada uma das partes do modelo
    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['relations'] = relations
    model['normals'] = normals

    return model

### Associando textura a um id

In [51]:
#Função: associa id com a textura
#Entradas: o id que queremos associar e o caminho do arquivo .jpg
#Saida: não possui, apenas associa
def load_texture_from_file(texture_id, img_textura):
    #Definindo o id
    glBindTexture(GL_TEXTURE_2D, texture_id)
    #Alterando configurações paramétricas de textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    #Abre imagem
    img = Image.open(img_textura)
    #Captura as dimensões
    img_width = img.size[0]
    img_height = img.size[1]
    #Transforma imagem para um sequência de bytes em formato raw de arquivo
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #Carregando os dados da imagem
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Pega informações de coordenadas de vértice, textura e normal

In [52]:
# Função: retorna uma lista para coordenadas dos vertices do objeto, outra para as da textura e outra para as das normais
def get_vertices_textures_normals(model):
    vertices_list = []    
    textures_coord_list = []
    normals_list = []
    # Para cada um das relations (num_line(v), num_line(vt), num_line(vn) material)
    for relation in model['relations']:
        # Para cada um dos números que representa a linha do vértice
        for vertice_id in relation[0]: # Pega o valor a coordenada do vértice
            vertices_list.append( model['vertices'][vertice_id-1] )
        # Para cada um dos números que representa a linha da coordenada da textura
        for texture_id in relation[1]:  # Pega o valor a coordenada da textura
            textures_coord_list.append( model['texture'][texture_id-1] )
        # Para cada um dos números que representa a linha da coordenada da normal
        for normal_id in relation[2]:
            normals_list.append( model['normals'][normal_id-1] )
    return vertices_list, textures_coord_list, normals_list

### Fazendo carregamento

In [53]:
#Ativando texturas 2D
glEnable(GL_TEXTURE_2D)

#Gerando ids
num_textures = 10
textures = glGenTextures(num_textures)

#Carregando modelos
path_wave = 'objetos_wavefront'
# Os nomes devem ser iguais a da pasta
names_obj = ['caixa', 'luz', 'luz']
id_obj = 0
objs_wave = []
for name_obj in names_obj:
    path_obj = f'{path_wave}\{name_obj}\{name_obj}.obj'
    path_jpg = f'{path_wave}\{name_obj}\{name_obj}.jpg'
    if name_obj == 'luz':
        path_jpg = f'{path_wave}\{name_obj}\{name_obj}.png'
    id_obj = id_obj + 1
    objs_wave.append((id_obj, path_obj, path_jpg))

In [54]:
# Quantidade total de vértices de objeto a serem utilizados neste programa
total_len_vert_obj = 0
# Quantidade total de vértices de textura a serem utilizados neste programa
total_len_vert_text = 0
# Quantidade total de normais a serem utilizados neste programa
total_len_normals = 0
# Vértices de objeto a serem utilizados neste programa
vertices_obj_total = []
# Vértices de textura a serem utilizados neste programa
vertices_text_total = []
# Normais a serem utilizados neste programa
normals_total = []
# Controla os valores que definem a identificação de cada objeto
identification = []

# Para cada objeto:
for obj in objs_wave:
    # Captura o modelo
    modelo = load_model_from_file(obj[1])
    # Carrega a textura
    load_texture_from_file(obj[0], obj[2])
    # Captura os vertices do objeto e de textura
    vertices_list, textures_coord_list, normals_list = get_vertices_textures_normals(modelo)
    # Declara a identificação
    identification.append((total_len_vert_obj, len(vertices_list)))
    # Adiciona o tamanho total dos vertices de objeto e de textura
    total_len_vert_obj += len(vertices_list)
    total_len_vert_text += len(textures_coord_list)
    total_len_normals += len(normals_list)
    # Adiciona os vértices de objeto e de textura
    vertices_obj_total += vertices_list
    vertices_text_total += textures_coord_list
    normals_total += normals_list

#Finaliza a modelagem dos dados de vértices
vertices = np.zeros(total_len_vert_obj, [("position", np.float32, 3)])
vertices['position'] = vertices_obj_total

#Finaliza a modelagem dos dados de texturas
textures = np.zeros(total_len_vert_text, [("position", np.float32, 2)])
textures['position'] = vertices_text_total

#Finaliza a modelagem dos dados das normais
normals = np.zeros(total_len_normals, [("position", np.float32, 3)])
normals['position'] = normals_total

### Manipulação dos espaços de dados

In [55]:
#Solicita dois buffers para GPU
buffer = glGenBuffers(3)

# Enviando dados de vértice
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

# Enviando dados de textura
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
#Subindo os dados de textura para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das texturas
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "texture_coord" e habilitando
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
#Linkando dados ao atributo "texture_coord"
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


# Enviando dados das normais
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
#Subindo os dados das normais para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das normais
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "normals" e habilitando
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
#Linkando dados ao atributo "normals"
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Definindo funções para matriz Model, View e Projection

In [56]:
# Matriz model: posição, orientação e escala do objeto no próprio espaço local
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    # Instanciando uma matriz identidade para ser alterada
    model_matrix = glm.mat4(1.0) 

    # ROTAÇÃO
    angle = math.radians(angle) # ângulo de rotação
    # r_x, r_y, r_z: flags indicando quais eixos deverão ser rotacionados
    model_matrix = glm.rotate(model_matrix, angle, glm.vec3(r_x, r_y, r_z))

    # ESCALA
    model_matrix = glm.scale(model_matrix, glm.vec3(s_x, s_y, s_z))

    # TRANSLAÇÃO
    model_matrix = glm.translate(model_matrix, glm.vec3(t_x, t_y, t_z))
    
    # TRANSPOSTA: glm trabalha com ela invertida
    model_matrix = np.array(model_matrix).T
    
    return model_matrix

# Matriz view: posição e orientação da câmera no espaço 3D. Posiciona a cena em relação à câmera.
def view():
    global cameraPos, cameraFront, cameraUp
    # Parãmetros: posição da câmera, direção do target e câmera up. Câmera direita é calculado internamente pela função da biblioteca glm
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
    mat_view = np.array(mat_view)
    return mat_view

# Matriz Projection: transforma o volume de visualização 3D em um espaço 2D, levando em consideração fatores como a distância dos objetos à câmera
def projection():
    # Neste caso, definimos parâmetros estáticos, mas poderiam ser dinâmicos
    fov = glm.radians(45.0)
    aspect = WIDTH_WINDOW/HEIGHT_WINDOW
    near = 0.1
    far = 1000.0
    mat_projection = glm.perspective(fov, aspect, near, far)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Eventos de mouse e teclado

In [57]:
# Posicao inicial da camera
cameraPos   = glm.vec3(0.0,  0.0,  15.0)
# Vetor responsável para apontar para frente
cameraFront = glm.vec3(0.0,  0.0, -1.0)
# Vetor auxiliar que aponta para cima em relação a camera
cameraUp    = glm.vec3(0.0,  1.0,  0.0)

# Definindo se haverá textura ou não
polygonal_mode = False

# Angulo da luz
ang = 0.1
# Valor do expoente de reflexão especular
ns_inc = 32

# Funcao que captura evento do teclado
def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, ns_inc
    
    # Componentes da câmera
    # Velocidade da camera
    cameraSpeed = 0.05
    # Ir para frente
    if key == 87 and (action==1 or action==2): # Tecla W
        cameraPos += cameraSpeed * cameraFront
    # Ir para trás
    if key == 83 and (action==1 or action==2): # Tecla S
        cameraPos -= cameraSpeed * cameraFront
    # Ir para esquerda
    if key == 65 and (action==1 or action==2): # Tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    # Ir para direita    
    if key == 68 and (action==1 or action==2): # Tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    # Alterna ativação da textura
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    elif key == 80 and action==1 and polygonal_mode==False:
        polygonal_mode=True
    
    # Aumenta o expoente de reflexão especular multiplicando por 2
    if key == 265 and (action==1 or action==2): # Tecla UP
        ns_inc = ns_inc * 2
    # Diminuir o expoente de reflexão especular dividindo por 2
    if key == 264 and (action==1 or action==2): # Tecla DOWN
        ns_inc = ns_inc / 2

# Variáveis auxiliar
# Flag para definir se eh a primeira vez que o mouse aparece na tela
firstMouse = True
# yaw: rotação no eixo y
yaw = -90.0 
# pitch: rotação no eixo x
pitch = 0.0
# Valores iniciais da última posição do mouse
lastX =  WIDTH_WINDOW/2
lastY =  HEIGHT_WINDOW/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    # Tratando caso de primeira aparição do mouse
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    # Calculos da variação
    xoffset = xpos - lastX
    yoffset = lastY - ypos
    # Atualizando valor da última posição
    lastX = xpos
    lastY = ypos
    # Calculando yam e pitch aproximadamente
    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity
    yaw += xoffset
    pitch += yoffset

    # Evitando que rotação extremas
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    # Fórmulas matemáticas para calcular o novo cameraFront
    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# Define função de evento para teclado
glfw.set_key_callback(window,key_event)
# Define função de evento para cursor
glfw.set_cursor_pos_callback(window, mouse_event)
# Seta posição do cursor
# glfw.set_cursor_pos(window, lastX, lastY)

### Funções de Impressão de Objeto

In [58]:
def desenha_caixa(init, len_vertices, id):
    # MATRIZ MODEL
    # Rotação
    angle = 0.0                     # ângulo de rotação    
    r_x = 0.0; r_y = 1.0; r_z = 0.0 # definindo quais eixos serão rotacionados
    # Translacao
    t_x = 0.0; t_y = 0.0; t_z = 0.0
    # Escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0
    # Recebendo matriz model
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    # Capturando localização do qualificador
    loc_model = glGetUniformLocation(program, "model")  
    # Enviando matriz model
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    # ILUMINAÇÃO
    # Definindo parâmetros iluminação
    ka = 0.1 # coeficiente de reflexao ambiente do modelo
    kd = 0.5 # coeficiente de reflexao difusa do modelo
    ks = 0.9 # coeficiente de reflexao especular do modelo
    ns = 64.0 # expoente de reflexao especular
    
    # Capturando localização dos qualificadores
    loc_ka = glGetUniformLocation(program, "ka")
    loc_kd = glGetUniformLocation(program, "kd") 
    loc_ks = glGetUniformLocation(program, "ks")
    loc_ns = glGetUniformLocation(program, "ns")

    # Enviando parâmetros para GPU
    glUniform1f(loc_ka, ka) 
    glUniform1f(loc_kd, kd)   
    glUniform1f(loc_ks, ks)
    glUniform1f(loc_ns, ns)
    
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, id)    
    
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, init, len_vertices)

def desenha_luz(init, len_vertices, id, t_x, t_y, t_z, num_luz = 1):
    # MATRIZ MODEL
    # Rotação
    angle = 0.0                     # ângulo de rotação    
    r_x = 0.0; r_y = 0.0; r_z = 1.0 # definindo quais eixos serão rotacionados
    # Translacao: dado como parâmetro
    # t_x = 0.0; t_y = 0.0; t_z = 0.0
    # Escala
    s_x = 0.1; s_y = 0.1; s_z = 0.1
    # Recebendo matriz model
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    # Capturando localização do qualificador
    loc_model = glGetUniformLocation(program, "model")  
    # Enviando matriz model
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    # ILUMINAÇÃO
    # Definindo parâmetros iluminação
    ka = 1 # coeficiente de reflexao ambiente do modelo
    kd = 1 # coeficiente de reflexao difusa do modelo
    ks = 1 # coeficiente de reflexao especular do modelo
    ns = 1000.0 # expoente de reflexao especular
    
    # Capturando localização dos qualificadores
    loc_ka = glGetUniformLocation(program, "ka")
    loc_kd = glGetUniformLocation(program, "kd") 
    loc_ks = glGetUniformLocation(program, "ks")
    loc_ns = glGetUniformLocation(program, "ns")

    # Enviando parâmetros para GPU
    glUniform1f(loc_ka, ka) 
    glUniform1f(loc_kd, kd)   
    glUniform1f(loc_ks, ks)
    glUniform1f(loc_ns, ns)

    # Adicional: informando localização da posição da luz
    if num_luz == 1:
        loc_light_pos = glGetUniformLocation(program, "lightPos1")
    else:
        loc_light_pos = glGetUniformLocation(program, "lightPos2")
    glUniform3f(loc_light_pos, t_x, t_y, t_z)

    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, id)    
    
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, init, len_vertices)

### Exibindo na tela

In [59]:
glfw.show_window(window)

## Loop principal

In [60]:
# Habilita 3D
glEnable(GL_DEPTH_TEST)
    
while not glfw.window_should_close(window):
    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(0.2, 0.2, 0.2, 1.0)

    if polygonal_mode:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    else:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)

    # Desenhando objeto
    desenha_caixa(identification[0][0], identification[0][1], 0)   
    
    # Desenhando fonte de luz
    ang += 0.01
    desenha_luz(identification[1][0], identification[1][1], 1, np.sin(ang)*35.0, 0.0, np.cos(ang)*35.0, 1)  
    desenha_luz(identification[2][0], identification[2][1], 2, np.cos(ang)*35.0, np.sin(ang)*35.0, 0.0, 2)   
    
    # Computando e enviando matrizes (model já foi enviada)
    # View
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)
    # Projection
    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)     
    
    # Atualizando a posicao da câmera/observador na GPU para cálculo da reflexão especular
    loc_view_pos = glGetUniformLocation(program, "viewPos")
    glUniform3f(loc_view_pos, cameraPos[0], cameraPos[1], cameraPos[2])
    
    glfw.swap_buffers(window)

glfw.terminate()